<a href="https://colab.research.google.com/github/Dineshrestha/ArcGIS-API-for-Python/blob/master/Intersect_Features_and_Export_all_Excels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prompt
Here is a work flow that we want to automate:


1.   Buffer all features (differently) based on the buffer distance listed in the excel sheet
2.   Loop through the buffered feature classes, then intersect
3. After intersection, export the results in excel files.



### Setting up the environment:
- First, you need to import the arcpy module to access the ArcGIS Pro functionality.
- Then, you can specify the necessary paths and folders for the workflow.

## Geodatabase paths and folders:
- You need to specify the input geodatabase path (input_geodatabase) where the original feature classes are stored.
- Similarly, you specify the output geodatabase path (output_geodatabase) where the buffered feature classes will be saved.
- Additionally, you define the intersect output geodatabase path (intersect_output_geodatabase) where the intersected feature classes will be stored.
- You also specify the folder path for the output spreadsheets (output_spreadsheets_folder) where the attribute tables will be exported.


## Buffering and intersecting:
- The code uses arcpy to perform buffering and intersecting operations.
- It loops through the feature classes in the input geodatabase and checks if they exist.
- For each existing feature class, it buffers the features based on the specified buffer distance and saves the buffered feature class to the output geodatabase.
- Then, it intersects the buffered features with all the other features in the intersect output geodatabase and saves the intersected feature class to the intersect output geodatabase.

## Importing and looping through the spreadsheet:
- The code imports the Excel spreadsheet using the pandas library and reads the data from the specified sheet name ("Intersecting_Layers").
- It loops through each row in the spreadsheet and extracts the feature names and buffer distances.
- For each feature name and buffer distance, it checks if the feature class exists in the input geodatabase.
- If the feature class exists, it performs the buffer operation with the specified buffer distance.

## Exporting attribute tables to spreadsheets:
- After the buffer and intersect operations, the code sets the workspace to the intersect output geodatabase.
- It retrieves a list of feature classes in the intersect output geodatabase.
- Then, it iterates through each feature class and exports the attribute table to a separate spreadsheet in the specified output spreadsheets folder.

## Printing the progress and completion:
- Throughout the workflow, the code includes print statements to provide feedback on the progress.
- It prints messages when skipping non-existing feature classes, buffering features, intersecting features, and exporting attribute tables.
- Finally, it prints a message indicating the completion of the export process.

In [ ]:
import arcpy
import os
import pandas as pd
import datetime

# Get the current time as the start time
start_time = datetime.datetime.now()

# Your code for the processing goes here

# Get the current time as the end time
end_time = datetime.datetime.now()

# Calculate the total processing time
total_time = end_time - start_time

# Print the start time, end time, and total processing time
print("Start Time: ", start_time)
print("End Time: ", end_time)
print("Total Processing Time: ", total_time)


# Set up the environment
arcpy.env.overwriteOutput = True

# Specify the input geodatabase path
input_geodatabase = r"Z:\Private\dshrestha\Projects\Sunzia\Intersect_20230607\Intersect_Layers.gdb"

# Specify the output geodatabase path
output_geodatabase = r"Z:\Private\dshrestha\Projects\Sunzia\Intersect_20230607\Buffer_Layers.gdb"

# Specify the intersect output geodatabase path
intersect_output_geodatabase = r"Z:\Private\dshrestha\Projects\Sunzia\Intersect_20230607\Buffer_Intersect_Layers.gdb"

# Specify the spreadsheet path
spreadsheet_path = r"Z:\Private\dshrestha\Projects\Sunzia\Intersect_20230607\Spreadsheet\LayerBufferInterceptRequest_dshrestha.xlsx"

# Specify the output spreadsheets folder
output_spreadsheets_folder = r"Z:\Private\dshrestha\Projects\Sunzia\Intersect_20230607\OutputSpreadsheets"

# Create the output spreadsheets folder if it doesn't exist
if not os.path.exists(output_spreadsheets_folder):
    os.makedirs(output_spreadsheets_folder)
    print("Output spreadsheets folder created successfully.")
else:
    print("Output spreadsheets folder already exists.")



# Create the output geodatabase if it doesn't exist
if not arcpy.Exists(output_geodatabase):
    arcpy.CreateFileGDB_management(os.path.dirname(output_geodatabase), os.path.basename(output_geodatabase))

# Create the intersect output geodatabase if it doesn't exist
if not arcpy.Exists(intersect_output_geodatabase):
    arcpy.CreateFileGDB_management(os.path.dirname(intersect_output_geodatabase), os.path.basename(intersect_output_geodatabase))

# Import the spreadsheet into a pandas DataFrame
spreadsheet_df = pd.read_excel(spreadsheet_path, sheet_name="Intersecting_Layers")

# Loop through each row in the DataFrame
for index, row in spreadsheet_df.iterrows():
    data_layer = str(row['Data_Layer'])  # Convert to string
    buffer_meters = row['Buffer_Meters']

    # Check if the feature exists in the input geodatabase
    feature_path = None
    for dirpath, dirnames, filenames in arcpy.da.Walk(input_geodatabase, datatype="FeatureClass"):
        for filename in filenames:
            if filename[:-9].lower() in data_layer.lower():
                feature_path = os.path.join(dirpath, filename)
                break
        if feature_path:
            break

    # If the feature doesn't exist, skip to the next row
    if not feature_path:
        print(f"Feature '{data_layer}' does not exist in the input geodatabase. Skipping...")
        continue

    # Get the feature name from Data_Layer by removing the last 9 characters (i.e., "_yyyyMMdd")
    feature_name = os.path.splitext(os.path.basename(feature_path))[0][:-9]

    # Buffer the feature if Buffer_Meters is greater than 0
    if buffer_meters > 0:
        buffer_output_name = feature_name + "_buffer"
        buffer_output_path = os.path.join(output_geodatabase, buffer_output_name)
        arcpy.Buffer_analysis(feature_path, buffer_output_path, buffer_meters)
        print(f"Buffer created for feature '{feature_name}' with buffer distance of {buffer_meters} meters.")

        # Perform intersection analysis
        intersect_output_name = feature_name + "_intersect"
        intersect_output_path = os.path.join(intersect_output_geodatabase, intersect_output_name)
        arcpy.Intersect_analysis([feature_path, buffer_output_path], intersect_output_path)
        print(f"Intersection analysis completed for feature '{feature_name}'.")

        # Export attribute table to a spreadsheet
        output_spreadsheet_name = feature_name + ".xlsx"
        output_spreadsheet_path = os.path.join(output_spreadsheets_folder, output_spreadsheet_name)
        arcpy.TableToExcel_conversion(intersect_output_path, output_spreadsheet_path)
        print(f"Exported attribute table for feature '{feature_name}' to spreadsheet.")

    else:
        print(f"Could not run buffer for feature '{feature_name}' since Buffer_Meters = 0.")

print("Process complete.")